In [1]:
import os
import torch
import numpy as np
import time
import random
from sklearn.model_selection import KFold
import argparse
import timeit
from dataset import prepare_data
import psutil
import torch
from models.gconvGRU import GConvGRUModel
from memory_capacity_utils import gen_lag_data_embeddings, get_mem_cap_from_model

In [2]:
if torch.cuda.is_available():
    device = torch.device('cuda:0')
    print('running on GPU')
else:
    device = torch.device("cpu")
    print('running on CPU')


running on GPU


# Load the dataset

In [3]:
# dataset = np.load('../oasis_data.npy')
# dataset = torch.from_numpy(dataset).squeeze()
# dataset = dataset.type(torch.FloatTensor)
# dataset = np.delete(dataset,88,axis=0)

dataset = np.load("datasets/multivariate_simulation_data_2.npy")
dataset = torch.from_numpy(dataset).squeeze()
dataset = dataset.type(torch.FloatTensor)

In [4]:
def get_args():
    parser = argparse.ArgumentParser(description='Args for graph predition')
    parser.add_argument('-num_folds', type=int, default=5, help='cv number')
    parser.add_argument('--num_timepoints', type=int, default=3,
                        help='Number of timepoints')
    parser.add_argument('-num_epochs', type=int, default=20, help='number of epochs')
    parser.add_argument('--lr', type=float, default=0.0001, help="Learninng rate")
    parser.add_argument('--memcap_coef', type=float, default=0, help="Memory Capacity Loss Coefficient")
    parser.add_argument('-max_lag', type=int, default=35, help='Lag tao for memory capacity signals')
    parser.add_argument('-save_path',type=str,default = '/vol/bitbucket/sx420/4D-FedGNN-Plus/results/baseline_gconvGRU_language/',help='Path to the saved results')
    args, _ = parser.parse_known_args()
    return args

In [5]:
args = get_args()

# Check

# Set up the network architecture and train-validate

**Train-validate-functions**

In [6]:
manual_seed = 777
np.random.seed(manual_seed)
random.seed(manual_seed)
torch.manual_seed(manual_seed)

embeddings = np.load('datasets/gutenberg_embeddings.npy')
# Reservoir signals
X_train_res_np, y_train_res_np = gen_lag_data_embeddings(embeddings, 1000, args.max_lag, 41)
X_test_res_np, y_test_res_np = gen_lag_data_embeddings(embeddings, 500, args.max_lag, 42)
X_train_res = torch.from_numpy(X_train_res_np).unsqueeze(1).to(device, dtype=torch.float64)
X_test_res = torch.from_numpy(X_test_res_np).unsqueeze(1).to(device, dtype=torch.float64)
y_train_res = torch.from_numpy(y_train_res_np).to(device, dtype=torch.float64)
y_test_res = torch.from_numpy(y_test_res_np).to(device, dtype=torch.float64)

In [7]:
def create_directory_if_not_exists(directory):
    """
    Checks if a specified directory exists, and creates it if it doesn't.

    Args:
    - directory (str): Path of the directory to check and potentially create.
    """
    if not os.path.exists(directory):
        os.makedirs(directory)
        print(f"Directory '{directory}' was created.")
    else:
        print(f"Directory '{directory}' already exists.")

# Create the results folders
create_directory_if_not_exists(args.save_path+'real_and_predicted_graphs')
create_directory_if_not_exists(args.save_path+'train_losses/mae_losses')
create_directory_if_not_exists(args.save_path+'train_losses/reservoir_losses')
create_directory_if_not_exists(args.save_path+'train_losses/bio_losses')
create_directory_if_not_exists(args.save_path+'train_losses/tp_losses')
create_directory_if_not_exists(args.save_path+'train_losses/total_losses')
create_directory_if_not_exists(args.save_path+'test_mae_losses')
create_directory_if_not_exists(args.save_path+'test_tp_losses')
create_directory_if_not_exists(args.save_path+'test_memcap_losses')
create_directory_if_not_exists(args.save_path+'test_predicted')
create_directory_if_not_exists(args.save_path+'test_original')
create_directory_if_not_exists(args.save_path+'trained_models')

Directory '/vol/bitbucket/sx420/4D-FedGNN-Plus/results/baseline_gcLSTM/real_and_predicted_graphs' already exists.
Directory '/vol/bitbucket/sx420/4D-FedGNN-Plus/results/baseline_gcLSTM/train_losses/mae_losses' already exists.
Directory '/vol/bitbucket/sx420/4D-FedGNN-Plus/results/baseline_gcLSTM/train_losses/reservoir_losses' already exists.
Directory '/vol/bitbucket/sx420/4D-FedGNN-Plus/results/baseline_gcLSTM/train_losses/bio_losses' already exists.
Directory '/vol/bitbucket/sx420/4D-FedGNN-Plus/results/baseline_gcLSTM/train_losses/tp_losses' already exists.
Directory '/vol/bitbucket/sx420/4D-FedGNN-Plus/results/baseline_gcLSTM/train_losses/total_losses' already exists.
Directory '/vol/bitbucket/sx420/4D-FedGNN-Plus/results/baseline_gcLSTM/test_mae_losses' already exists.
Directory '/vol/bitbucket/sx420/4D-FedGNN-Plus/results/baseline_gcLSTM/test_tp_losses' already exists.
Directory '/vol/bitbucket/sx420/4D-FedGNN-Plus/results/baseline_gcLSTM/test_memcap_losses' already exists.
Direc

In [8]:
def validation(model, validation_subjects, mem_cap_data, X_train, y_train, X_test, y_test):
    mael = torch.nn.L1Loss().to(device)
    tp = torch.nn.MSELoss().to(device)
    
    val_mae_loss = np.zeros(args.num_timepoints - 1)
    val_tp_loss = np.zeros(args.num_timepoints - 1)
    mem_cap = np.zeros(args.num_timepoints - 1)
    predicted = np.zeros((validation_subjects.shape[0], args.num_timepoints - 1, 35, 35))
    actual = np.zeros((validation_subjects.shape[0], args.num_timepoints - 1, 35, 35))
    
    model.eval()

    with torch.no_grad():
        for n_subject, data in enumerate(validation_subjects):
            input = data[0]
            for t in range(args.num_timepoints - 1):
                pred = model(input)
                val_mae_loss[t] += mael(pred, data[t + 1])
                val_tp_loss[t] += tp(pred.sum(dim=-1), data[t + 1].sum(dim=-1))
                input = pred
                
                pred_mem_cap = get_mem_cap_from_model(model, pred, 
                                                      X_train, y_train, X_test, y_test)
                actual_mem_cap = torch.tensor(mem_cap_data[n_subject, t + 1]).to(device)
                mem_cap[t] += torch.abs(pred_mem_cap - actual_mem_cap)

                predicted[n_subject, t] = pred.cpu().detach().numpy()
                actual[n_subject, t] = data[t + 1].cpu().detach().numpy()
                

    avg_val_mae_loss = val_mae_loss/len(validation_subjects)
    avg_val_tp_loss = val_tp_loss/len(validation_subjects)
    avg_val_mae_mem_cap = mem_cap/len(validation_subjects)

    return avg_val_mae_loss, avg_val_tp_loss, avg_val_mae_mem_cap, predicted, actual

In [12]:
torch.manual_seed(1)
input_weights = (torch.rand((35, 1), dtype=torch.float64) * 2.0 - 1.0).to(device)

indexes = range(args.num_folds)
kfold = KFold(n_splits=args.num_folds, shuffle=True, random_state=manual_seed)
dataset = dataset.to(device)
actual_mem_caps = np.load('datasets/mem_caps_2_language.npy')
f = 0

for train, test in kfold.split(range(dataset.shape[0])):
    print(
            f'------------------------------------Fold [{f + 1}/{args.num_folds}]-----------------------------------------')
    
    train_data = dataset[train]
    test_data = dataset[test]
    train_mem_cap = actual_mem_caps[train]
    test_mem_cap = actual_mem_caps[test]
    
    validation_split = int(0.8 * len(train_data))
    train_subjects = train_data[:validation_split]
    train_mem_cap_subjects = train_mem_cap[:validation_split]
    validation_subjects = train_data[validation_split:]
    validation_mem_cap_subjects = train_mem_cap[:validation_split]

    # Create model instance
    model = GConvGRUModel(device=device, input_weights=input_weights, input_scaling=1e-1).to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=args.lr)
    mael = torch.nn.L1Loss().to(device)
    tp = torch.nn.MSELoss().to(device)
  
    # Start measuring the epochs time
    epochs_start = time.time()
    for epoch in range(args.num_epochs):
        
        print(f'Epoch [{epoch + 1}/{args.num_epochs}]')
        # Set the model in training mode
        model.train()
        
        # this is our loss for all the data
        mae_loss_overall = []
        tp_loss_overall = []
        
        # loop through the data batches
        for data_id, data in enumerate(train_subjects):

            # zero the gradients
            optimizer.zero_grad()
            mae_loss = 0
            tp_loss = 0
         
            # loop through the time dependent adj matrices in the batches
            for t in range(args.num_timepoints - 1): 
                pred = model(data[t])
                real = data[t + 1]
                # print(pred)
                mae_loss += mael(pred, real)

                # Topological Loss
                tp_loss += tp(pred.sum(dim=-1), real.sum(dim=-1))

    
            # Calculate the total MAE Loss for the current batch
            mae_loss = mae_loss / (args.num_timepoints - 1)

            # Calculate the total TP Loss for the current batch
            tp_loss = tp_loss / (args.num_timepoints - 1)

            # Append to the total MAE Loss
            mae_loss_overall.append(mae_loss.item())
            tp_loss_overall.append(tp_loss.item())
            
            total_loss = mae_loss
            # Update the weights of the neural network
            total_loss.backward()
            optimizer.step()

        mae_loss_overall = np.mean(np.array(mae_loss_overall))
        tp_loss_overall = np.mean(np.array(tp_loss_overall))
        print(f"[Train] MAE Loss: {mae_loss_overall}, TP Loss: {tp_loss_overall}")
    
        avg_val_mae_loss, avg_val_tp_loss, avg_val_mae_mem_cap, _, _ = validation(model, validation_subjects, validation_mem_cap_subjects,
                                                                                  X_train_res, y_train_res, X_test_res, y_test_res)
        print(f"[Validate] MAE Loss Across Timepoints: {avg_val_mae_loss}")
        print(f"[Validate] TP Loss Across Timepoints: {avg_val_tp_loss}")
        print(f"[Validate] MAE of Mem Caps Across Timepoints: {avg_val_mae_mem_cap}")

    
    epochs_end = time.time() - epochs_start
    print()
    print(f'epochs finished with time:{epochs_end}')
    print()
    process = psutil.Process(os.getpid())
    print(f"Current memory usage: {process.memory_info().rss / 1024 ** 2:.2f} MB")
    print()
    
    avg_test_mae_loss, avg_test_tp_loss, avg_test_mem_cap, predicted, original = validation(model, test_data, test_mem_cap,
                                                                                            X_train_res, y_train_res, X_test_res, y_test_res)
    print(f"[Test] MAE Loss Across Timepoints: {avg_test_mae_loss}")
    print(f"[Test] TP Loss Across Timepoints: {avg_test_tp_loss}")
    print(f"[Test] MAE of Mem Caps Across Timepoints: {avg_test_mem_cap}")
    np.save(args.save_path+f"test_mae_losses/mae_test_loss_fold_{f}", avg_test_mae_loss)
    np.save(args.save_path+f"test_tp_losses/tp_test_loss_fold_{f}", avg_test_mae_loss)
    np.save(args.save_path+f"test_memcap_losses/memcap_test_loss_fold_{f}", avg_test_mem_cap)
    np.save(args.save_path+f"test_predicted/predicted_fold_{f}", predicted)
    np.save(args.save_path+f"test_original/original_fold_{f}", original)

    torch.save(model.state_dict(),
               args.save_path +f'trained_models/model_fold_{f}')
    f += 1
    

------------------------------------Fold [1/5]-----------------------------------------
Epoch [1/20]
[Train] MAE Loss: 0.041346488607814535, TP Loss: 2.3371906960383058
[Validate] MAE Loss Across Timepoints: [0.04590788 0.05016259]
[Validate] TP Loss Across Timepoints: [1.17481446 1.6991843 ]
[Validate] MAE of Mem Caps Across Timepoints: [0.73323912 1.48277225]
Epoch [2/20]
[Train] MAE Loss: 0.028385661222273484, TP Loss: 0.7183135610539466
[Validate] MAE Loss Across Timepoints: [0.03459761 0.0354607 ]
[Validate] TP Loss Across Timepoints: [0.91304427 1.15548646]
[Validate] MAE of Mem Caps Across Timepoints: [1.75001271 2.02549011]
Epoch [3/20]
[Train] MAE Loss: 0.02543649218569044, TP Loss: 0.655409298138693
[Validate] MAE Loss Across Timepoints: [0.03386762 0.0348905 ]
[Validate] TP Loss Across Timepoints: [0.87111574 1.10316646]
[Validate] MAE of Mem Caps Across Timepoints: [0.99788789 1.05861295]
Epoch [4/20]
[Train] MAE Loss: 0.02469104107876774, TP Loss: 0.6268129523377866
[Valid

[Validate] MAE Loss Across Timepoints: [0.03253255 0.0340913 ]
[Validate] TP Loss Across Timepoints: [0.85487193 1.0270617 ]
[Validate] MAE of Mem Caps Across Timepoints: [0.94025923 1.10204481]
Epoch [10/20]
[Train] MAE Loss: 0.023082637519109994, TP Loss: 0.5843747139442712
[Validate] MAE Loss Across Timepoints: [0.03253138 0.03405628]
[Validate] TP Loss Across Timepoints: [0.85872126 1.03144455]
[Validate] MAE of Mem Caps Across Timepoints: [0.94911675 1.10497623]
Epoch [11/20]
[Train] MAE Loss: 0.023030587210087106, TP Loss: 0.5856832597637549
[Validate] MAE Loss Across Timepoints: [0.03251737 0.03401599]
[Validate] TP Loss Across Timepoints: [0.86087114 1.03096735]
[Validate] MAE of Mem Caps Across Timepoints: [1.01071243 1.0981164 ]
Epoch [12/20]
[Train] MAE Loss: 0.02286619012011215, TP Loss: 0.5802995344856754
[Validate] MAE Loss Across Timepoints: [0.03242958 0.03391805]
[Validate] TP Loss Across Timepoints: [0.85158288 1.0190258 ]
[Validate] MAE of Mem Caps Across Timepoints:

[Train] MAE Loss: 0.022329080413328484, TP Loss: 0.541023432626389
[Validate] MAE Loss Across Timepoints: [0.03325804 0.0344383 ]
[Validate] TP Loss Across Timepoints: [0.86267209 1.06039357]
[Validate] MAE of Mem Caps Across Timepoints: [1.02961819 0.79337644]
Epoch [19/20]
[Train] MAE Loss: 0.022272701680776663, TP Loss: 0.5363331704866141
[Validate] MAE Loss Across Timepoints: [0.03326665 0.03449034]
[Validate] TP Loss Across Timepoints: [0.8689273  1.07260144]
[Validate] MAE of Mem Caps Across Timepoints: [1.0349199  0.95288363]
Epoch [20/20]
[Train] MAE Loss: 0.02230743745167274, TP Loss: 0.5423625697148964
[Validate] MAE Loss Across Timepoints: [0.03324124 0.0343751 ]
[Validate] TP Loss Across Timepoints: [0.86383152 1.0622648 ]
[Validate] MAE of Mem Caps Across Timepoints: [1.03254208 1.04741051]

epochs finished with time:494.5947012901306

Current memory usage: 2697.80 MB

[Test] MAE Loss Across Timepoints: [0.03147686 0.03394839]
[Test] TP Loss Across Timepoints: [0.77880387 

[Validate] MAE Loss Across Timepoints: [0.03219732 0.03450734]
[Validate] TP Loss Across Timepoints: [0.78871852 1.04886603]
[Validate] MAE of Mem Caps Across Timepoints: [1.05376936 0.92936932]
Epoch [6/20]
[Train] MAE Loss: 0.02321353532897774, TP Loss: 0.5591316969366744
[Validate] MAE Loss Across Timepoints: [0.03207136 0.03414758]
[Validate] TP Loss Across Timepoints: [0.77677017 1.01518583]
[Validate] MAE of Mem Caps Across Timepoints: [0.98492568 0.94750406]
Epoch [7/20]
[Train] MAE Loss: 0.02298994177544955, TP Loss: 0.5559555895160884
[Validate] MAE Loss Across Timepoints: [0.03199917 0.03406509]
[Validate] TP Loss Across Timepoints: [0.77509826 1.01192904]
[Validate] MAE of Mem Caps Across Timepoints: [0.99924944 0.92504098]
Epoch [8/20]
[Train] MAE Loss: 0.022852072710520588, TP Loss: 0.5473553922493011
[Validate] MAE Loss Across Timepoints: [0.03196625 0.03390761]
[Validate] TP Loss Across Timepoints: [0.77186394 0.99954873]
[Validate] MAE of Mem Caps Across Timepoints: [0.